In [303]:
import torch
import random
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from torch.autograd import Variable

In [31]:
def read_data():
    with open('D:\\BDT lecture notes\\5008(Social Computing)\\fb\\facebook_combined.txt','rb') as f:
        data = f.readlines()
    
    data = [bytes.decode(r).strip('\n') for r in data]
    data = [r.split(' ') for r in data]
    data = [[int(r[0]),int(r[1])] for r in data]
    data = [tuple(pair) for pair in data]
    
    return data

In [32]:
def produce_adj(data):
    all_node = []
    for pair in data:
        all_node.append(pair[0])
        all_node.append(pair[1])
    all_node = list(set(all_node))
    adj = np.zeros([len(all_node),len(all_node)])
    for pair in data:
        adj[pair[0],pair[1]] = 1
        adj[pair[1],pair[0]] = 1
    return adj

# 读取数据

In [211]:
data= read_data()
adj = produce_adj(data)  #包含全部数据的邻接矩阵

In [212]:
ix = np.array(list(range(adj.shape[0])))     

# 选取了列编号为1888的用户，其所有粉丝张成一个community

因为他粉丝最多，有需要也可以选其他人

In [213]:
l = list(ix * adj[:,1888])
l = [int(r) for r in l if r!=0]

In [251]:
community = adj[np.ix_(l,l)]

#community是这个圈子构成的社交网络的adjacency matrix

In [37]:
#data_sub = []
#for i in range(community.shape[0]):
#    for j in range(community.shape[1]):
#        if community[i,j]:
#            data_sub.append(tuple([l[i],l[j]]))

In [38]:
#G = nx.Graph()
#G.add_edges_from(data_sub)

In [374]:
Community = Variable(torch.tensor(community),requires_grad=True)
Ones_1n = torch.ones([1,Community.shape[0]]).type(torch.DoubleTensor)   #方便起见，先写好矩阵对应的1向量，常用
Ones_n1 = torch.ones([Community.shape[0],1]).type(torch.DoubleTensor)

# 七个统计量

In [343]:
#Clustering coefficient
triangle = (np.matmul(community,community)*community)
triangle = np.matmul(triangle,np.ones([triangle.shape[0],1])).T/2
degree = np.matmul(np.ones([1,community.shape[0]]),community)# + np.matmul(community,np.ones([community.shape[0],1])).T
degree_2 = degree * (degree-1)
clust_coef = 2*triangle/degree_2

In [375]:
#Clustering coefficient(Torch)
Triangle = (Community.mm(Community)).mul(Community)
Triangle = Triangle.mm(Ones_n1).T/2
Degree = Ones_1n.mm(Community)
Degree_2 = Degree.mul(Degree-1)
Clust_coef = 2 * Triangle/Degree_2

In [371]:
#Degrees
degree = np.matmul(np.ones([1,community.shape[0]]),community)

In [384]:
#Degree(Torch)
Degree = Ones_1n.mm(Community)

In [381]:
#average degree of neibourhood
degree = np.matmul(np.ones([1,community.shape[0]]),community)
neibour_degree = np.matmul(degree,community) * np.ones([1,community.shape[0]])/degree

In [382]:
#average degree of neibourhood(Torch)
Degree = Ones_1n.mm(Community)
Neibour_degree = (Degree.mm(Community)).mul(Ones_1n)/Degree

In [387]:
#average clustering coef of neibourhood
triangle = (np.matmul(community,community)*community)
triangle = np.matmul(triangle,np.ones([triangle.shape[0],1])).T/2
degree = np.matmul(np.ones([1,community.shape[0]]),community)# + np.matmul(community,np.ones([community.shape[0],1])).T
degree_2 = degree * (degree-1)
clust_coef = 2*triangle/degree_2
neibour_clust_coef = np.matmul(clust_coef,community)
neibour_aver_clust_coef = neibour_clust_coef * np.ones([1,community.shape[0]])/degree

In [388]:
#average clustering coef of neibourhood(Torch)
Triangle = (Community.mm(Community)).mul(Community)
Triangle = Triangle.mm(Ones_n1).T/2
Degree = Ones_1n.mm(Community)
Degree_2 = Degree.mul(Degree-1)
Clust_coef = 2 * Triangle/Degree_2
Neibour_clust_coef = Clust_coef.mm(Community)
Neibour_aver_clust_coef = Neibour_clust_coef.mul(Ones_1n)/Degree

In [236]:
# number of edges in the neighborhood
edge_ego = np.zeros([1,community.shape[0]])
for i in range(community.shape[0]):
    t = np.matmul(np.matmul(np.ones([1,community.shape[0]]),community[[i],:] * community * community[:,[i]]),np.ones([community.shape[0],1]))/community[[i],:].sum()
    edge_ego[0,i] = t

In [254]:
#number of outgoing edges from the neighborhood
mid = community.copy()
mid+=np.diag(np.ones([community.shape[0]]))
out_edge_neibour = np.zeros([1,community.shape[0]])
for i in range(mid.shape[0]):
    out_edge_neibour[0,i] = np.diag(mid - mid[[i],:] * mid * mid[:,[i]]).sum()

In [295]:
# number of neighbors of neighbors (not in neighborhood)
mid = community.copy()
mid+=np.diag(np.ones([mid.shape[0]]))
neibour_neibour = np.zeros([1,community.shape[0]])
for i in range(mid.shape[0]):
    neibour = mid - mid[[i],:] * mid * mid[:,[i]]
    d = np.diag(neibour)
    neibour_neibour[0,i]=np.sum(d.reshape(1,-1)*neibour*d.reshape(-1,1))

In [283]:
tmp = np.array([[0,0,1,1,0],[0,0,1,1,1],[1,1,0,1,1],[1,1,1,0,1],[0,1,1,1,0]]).astype(float)
tmp+=np.diag(np.ones([tmp.shape[0]]))

In [284]:
neibour = tmp - tmp[[0],:] * tmp * tmp[:,[0]]
d = np.diag(neibour)
print(d.reshape(1,-1)*neibour*d.reshape(-1,1))


[[0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1.]]
